# Loading Libraries  

In [182]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

# Declaring the directory for the script and loading the datasets

In [183]:
username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin" or platform.system() == "Linux":
    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")

else:
    print("The OS is Windows")
    complePath = r"C:\Users\David\Documents\GitHub\SCGProject\Datasets\\"
    #pathwin = "C:\SCGProject\Datasets"
    #print("Complete path: ", pathwin)
    #dload = os.chdir(pathwin)
    dfCambio = pd.read_csv(
        complePath+"\CorrectedDatasets\dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"\CorrectedDatasets\dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "\CorrectedDatasets\dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"\CorrectedDatasets\dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets\dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"\CorrectedDatasets\dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"\CorrectedDatasets\dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"\CorrectedDatasets\dfVendite.csv")


Username:  David
The OS is Windows


# Creating the sales table using joins 

In [184]:
del dfVendite['Unnamed: 0']
dfVendite.head()

,NrMovimento,budget/cons,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA
0,21621,BUDGET,ART0001433,C00140,4,18.80
1,21629,BUDGET,ART0001610,C00140,12,145.28
2,21631,BUDGET,ART0003269,C00140,24,933.99
3,21633,BUDGET,ART0002249,C00140,1,36.33
4,21634,BUDGET,ART0002370,C00140,1,65.72


In [185]:
del dfClienti['Unnamed: 0']
del dfClienti['CodCondizioniPagam']
del dfClienti['FattCumulative']

dfClienti.head()

,Nr,Valuta
0,C00001,1
1,C00003,2
2,C00004,3
3,C00005,1
4,C00006,2


In [186]:
del dfCambio['Unnamed: 0']
dfCambio.head()

,CodiceValuta,Anno,TassoCambioMedio
0,1,BUDGET,1
1,2,BUDGET,"1,0541"
2,3,BUDGET,123.4
3,1,CONSUNTIVO,1
4,2,CONSUNTIVO,"1,1993"


## Join operations for the sales table

In [187]:
temp = sqldf(
    "SELECT  * FROM dfVendite ca JOIN dfClienti cl ON ca.NrOrigine = cl.Nr ")
del temp["Nr"]
temp.columns = ['NrMovimento', 'budget', 'NrArticolo', 'NrOrigine',
                'Quantity', 'ImportoVenditaValutaLocaleTOTALEVENDITA', 'Valuta']

# Renaming the variables 

temp.loc[temp.loc[:, "budget"] == "Consuntivo", "budget"] = "CONSUNTIVO"
temp.loc[temp.loc[:, "budget"] == "Budget", "budget"] = "BUDGET"

tot = sqldf(
    "SELECT * FROM temp t  JOIN dfCambio c ON t.Valuta = c.CodiceValuta AND t.budget = c.Anno ")

del tot["Valuta"]
del tot["Anno"]

tot.head()


,NrMovimento,budget,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,CodiceValuta,TassoCambioMedio
0,21621,BUDGET,ART0001433,C00140,4,18.80,2,"1,0541"
1,21629,BUDGET,ART0001610,C00140,12,145.28,2,"1,0541"
2,21631,BUDGET,ART0003269,C00140,24,933.99,2,"1,0541"
3,21633,BUDGET,ART0002249,C00140,1,36.33,2,"1,0541"
4,21634,BUDGET,ART0002370,C00140,1,65.72,2,"1,0541"


## Sales table for the final statement

In [188]:
salesFinal = sqldf(
    "SELECT * FROM tot t WHERE t.budget = 'Consuntivo' OR t.budget = 'CONSUNTIVO'")
salesFinal.head()


,NrMovimento,budget,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,CodiceValuta,TassoCambioMedio
0,35089,CONSUNTIVO,ART0000814,C00140,1,545.20,2,"1,1993"
1,35550,CONSUNTIVO,ART0000841,C00140,1,4650.13,2,"1,1993"
2,35894,CONSUNTIVO,ART0005136,C00483,1,1550.00,2,"1,1993"
3,35944,CONSUNTIVO,ART0000752,C00140,2,912.00,2,"1,1993"
4,35945,CONSUNTIVO,ART0000782,C00140,1,295.00,2,"1,1993"


## Sales table for the budget 

In [189]:
salesBudget = sqldf(
    "SELECT * FROM tot t WHERE t.budget = 'Budget' OR t.budget = 'BUDGET'")
salesBudget.head()


,NrMovimento,budget,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,CodiceValuta,TassoCambioMedio
0,21621,BUDGET,ART0001433,C00140,4,18.80,2,"1,0541"
1,21629,BUDGET,ART0001610,C00140,12,145.28,2,"1,0541"
2,21631,BUDGET,ART0003269,C00140,24,933.99,2,"1,0541"
3,21633,BUDGET,ART0002249,C00140,1,36.33,2,"1,0541"
4,21634,BUDGET,ART0002370,C00140,1,65.72,2,"1,0541"


# Creating the costs table using joins

In [190]:
del dfConsumi["Unnamed: 0"]

dfConsumi.columns = ['NrMovimento', 'budget', 'CodiceMP', 'NrArticolo',
                     'NrDocumento', 'QuantityMPImpiegata', 'ImportoCostoTOTALE']

dfConsumi.loc[dfConsumi.loc[:, "budget"] == "Consuntivo", "budget"] = "CONSUNTIVO"
dfConsumi.loc[dfConsumi.loc[:, "budget"] == "Budget", "budget"] = "BUDGET"
dfConsumi.head()

,NrMovimento,budget,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00


In [191]:
#Costo orario consuntivo
del dfCostoOrarioConsuntivo["Unnamed: 0"]
dfCostoOrarioConsuntivo


,Risorsa,AreaProduzione,CostoOrario
0,RIS1,A10,60
1,RIS1,A11,60
2,RIS1,A20,80
3,RIS1,A30,80
4,RIS1,A32,60
...,...,...,...
65,RIS9,A11,50
66,RIS9,A20,80
67,RIS9,A30,50
68,RIS9,A32,50


In [192]:
#Budget
del dfCostoOrarioBudget["Unnamed: 0"]
dfCostoOrarioBudget.head()

,Risorsa,AreaProduzione,CostoOrario
0,RIS1,A10,51.0
1,RIS1,A11,51.0
2,RIS1,A20,68.0
3,RIS1,A30,68.0
4,RIS1,A32,51.0


In [193]:
del dfImpiegoRisorse["Unnamed: 0"]
dfImpiegoRisorse.loc[dfImpiegoRisorse.loc[:, "budgetConsuntivo"] == "Consuntivo", "budgetConsuntivo"] = "CONSUNTIVO"
dfImpiegoRisorse.loc[dfImpiegoRisorse.loc[:, "budgetConsuntivo"] == "Budget", "budgetConsuntivo"] = "BUDGET"
dfImpiegoRisorse.head()

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,QuantitydiOutput
0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,2.0
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,2.0
2,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,2.0
3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.00,2.0
4,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.00,2.0


## Costs table for the final statement

In [194]:
a = sqldf("SELECT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO'")

b = sqldf("SELECT * FROM dfConsumi WHERE budget = 'CONSUNTIVO'")

tot = sqldf("SELECT distinct * FROM a AS a join b AS b ON a.NrArticolo = b.NrArticolo and a.NrOrdineProduzione = b.NrDocumento")

totConsuntivo = sqldf("""SELECT distinct NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, t.Risorsa, TempoRisorsa, CostoOrario as CostoOrarioRisorsa, QuantitydiOutput, 
NrMovimento as NrMovimentoMP, CodiceMP, QuantityMPImpiegata, ImportoCostoTOTALE as ImportoCostoTotaleMP
FROM tot AS t JOIN dfCostoOrarioConsuntivo AS c on t.Risorsa = c.Risorsa and t.NrAreaProduzione = c.AreaProduzione""")


totConsuntivo.head()


,NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,QuantitydiOutput,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP
0,ART0000018,ODP19-05998,Tornitura,A20,RIS2,1.0000,80,2.0,66819,ART0001074,30.0,5.15
1,ART0000018,ODP19-05998,Fresatura,A30,RIS5,1.0000,50,2.0,66819,ART0001074,30.0,5.15
2,ART0000018,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,2.0,66819,ART0001074,30.0,5.15
3,ART0000041,ODP19-05998,Tornitura,A20,RIS1,3.0000,80,4.0,66818,ART0005696,4.0,88.00
4,ART0000041,ODP19-05998,Fresatura,A30,RIS1,0.0001,80,4.0,66818,ART0005696,4.0,88.00


## Costs table for the budget

In [195]:
a = sqldf("SELECT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET'")

b = sqldf("SELECT * FROM dfConsumi WHERE budget = 'BUDGET'")

tot = sqldf("SELECT distinct * FROM a AS a join b AS b ON a.NrArticolo = b.NrArticolo and a.NrOrdineProduzione = b.NrDocumento")

totBudget = sqldf(
    """SELECT distinct NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, t.Risorsa, TempoRisorsa, CostoOrario as CostoOrarioRisorsa, QuantitydiOutput, 
NrMovimento as NrMovimentoMP, CodiceMP, QuantityMPImpiegata, ImportoCostoTOTALE as ImportoCostoTotaleMP 
    FROM tot AS t JOIN  dfCostoOrarioBudget AS c ON t.Risorsa = c.Risorsa and t.NrAreaProduzione = c.AreaProduzione""")

totBudget.head()


,NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,QuantitydiOutput,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP
0,ART0000018,ODP17-05342,Tornitura,A20,RIS2,1.00,68.00,2.0,28854,ART0001074,30.0,5.15
1,ART0000018,ODP17-05342,Fresatura,A30,RIS7,0.75,51.00,2.0,28854,ART0001074,30.0,5.15
2,ART0000018,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,2.0,28854,ART0001074,30.0,5.15
3,ART0000041,ODP17-05342,Tornitura,A20,RIS14,3.00,51.00,4.0,28853,ART0005696,4.0,92.00
4,ART0000041,ODP17-05342,Tornitura,A20,RIS14,1.50,51.00,0.0,28853,ART0005696,4.0,92.00


# Scostamento costi di materie prime

costo = volume produttivo * sum (impiego i-esimo * costo acquisizone i-esimo) 

che corriponde a 

costo = QuantitydiOutput * sum((ImportoCostoTotale/QuantityMPImpiegata) * QuantityMPImpiegata)

## Scelgiere come raggruppare

In [196]:
Amount_of_outputs = sqldf('''select distinct NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP, count(QuantitydiOutput) 
from totBudget group by NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP order by count(QuantitydiOutput) desc''')

Amount_of_outputs

,NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP,count(QuantitydiOutput)
0,ART0000116,ODP17-02128,Fresatura,A30,RIS9,1.50,42.50,27234,ART0005606,2.0,52.00,2
1,ART0000320,ODP17-10637,Fresatura,A30,RIS7,0.50,51.00,35511,ART0005409,75.0,25.41,2
2,ART0000628,ODP17-02528,Fresatura,A30,RIS9,9.00,42.50,25245,ART0005213,3905.0,320.19,2
3,ART0000630,ODP17-09327,Fresatura,A30,RIS9,4.00,42.50,32085,ART0005286,3444.0,106.53,2
4,ART0000644,ODP17-02392,Fresatura,A30,RIS9,6.75,42.50,25247,ART0005287,4200.0,126.42,2
...,...,...,...,...,...,...,...,...,...,...,...,...
3975,ART0005136,ODP17-15077,Controllo qualità,CQ,RIS16,0.00,12.75,34837,ART0005172,250.0,4.16,1
3976,ART0005136,ODP17-15077,Tornitura,A20,RIS14,1.25,51.00,34837,ART0005172,250.0,4.16,1
3977,ART0005136,ODP17-16230,Brocciatura / stozzatrice,A33,RIS13,0.25,42.50,35397,ART0005176,1.0,0.00,1
3978,ART0005136,ODP17-16230,Controllo qualità,CQ,RIS16,0.00,12.75,35397,ART0005176,1.0,0.00,1


In [197]:
sqldf('''select distinct NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP, QuantitydiOutput, count(*)
from totBudget group by NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP, QuantitydiOutput order by count(*) desc''')

,NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP,QuantitydiOutput,count(*)
0,ART0000018,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,28854,ART0001074,30.0,5.15,2.0,1
1,ART0000018,ODP17-05342,Fresatura,A30,RIS7,0.75,51.00,28854,ART0001074,30.0,5.15,2.0,1
2,ART0000018,ODP17-05342,Tornitura,A20,RIS2,1.00,68.00,28854,ART0001074,30.0,5.15,2.0,1
3,ART0000041,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,28853,ART0005696,4.0,92.00,4.0,1
4,ART0000041,ODP17-05342,Fresatura,A30,RIS7,1.25,51.00,28853,ART0005696,4.0,92.00,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4032,ART0005136,ODP17-15077,Controllo qualità,CQ,RIS16,0.00,12.75,34837,ART0005172,250.0,4.16,10.0,1
4033,ART0005136,ODP17-15077,Tornitura,A20,RIS14,1.25,51.00,34837,ART0005172,250.0,4.16,10.0,1
4034,ART0005136,ODP17-16230,Brocciatura / stozzatrice,A33,RIS13,0.25,42.50,35397,ART0005176,1.0,0.00,1.0,1
4035,ART0005136,ODP17-16230,Controllo qualità,CQ,RIS16,0.00,12.75,35397,ART0005176,1.0,0.00,1.0,1


# Scostamento costi di Risorsa

## Diversi tipi risorse

In [198]:
sqldf('select distinct Descrizione from totConsuntivo order by Descrizione')

,Descrizione
0,None
1,Brocciatura / stozzatrice
2,Controllo qualità
3,Fresatura
4,Fresatura (sbavatura fori)
5,Fresatura (sistemazione ore)
6,Montaggio
7,Piegatura
8,Preparazione materiale
9,Preparazione materiale/taglio/sbavatura
